#### 1. 假设我们输入矩阵
### [MarkDown math](https://www.zybuluo.com/codeep/note/163962)
$
M=\begin{pmatrix}
1 & -1 & 0 \\
-1 & 2 & 1 \\
0 & 2 & -2 \\
\end{pmatrix}
$

In [3]:
import tensorflow as tf
import numpy as np

M = np.array([
        [[1],[-1],[0]],
        [[-1],[2],[1]],
        [[0],[2],[-2]]
    ])

print "Matrix shape is: ",M.shape

Matrix shape is:  (3, 3, 1)


#### 2. 定义卷积过滤器, 深度为1。
$
W=
\begin{pmatrix}
1 & -1 \\
0 & 2 \\
\end{pmatrix}
$
```python
tf.get_variable(
    name,
    shape=None,
    dtype=None,
    initializer=None,
    regularizer=None,
    trainable=None,
    collections=None,
    caching_device=None,
    partitioner=None,
    validate_shape=True,
    use_resource=None,
    custom_getter=None,
    constraint=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.VariableAggregation.NONE
)
```

In [4]:
# shape: (2x2x1)x1 filters
filter_weight = tf.get_variable('weights', [2, 2, 1, 1], initializer = tf.constant_initializer([
                                                                        [1, -1],
                                                                        [0, 2]]))
# 
biases = tf.get_variable('biases', [1], initializer = tf.constant_initializer(1))

#### 3. 调整输入的格式符合TensorFlow的要求。

In [5]:
M = np.asarray(M, dtype='float32') # Convert the input to an array.
M = M.reshape(1, 3, 3, 1) # 

#### 4.  计算矩阵通过卷积层过滤器和池化层过滤器计算后的结果。

```python
tf.placeholder(
    dtype,
    shape=None,
    name=None
)
```

In [6]:
# shape=[batch_size, width, height, channel]
x = tf.placeholder('float32', [1, None, None, 1])
# tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
# strides=[1, 2, 2, 1],要求第一和最后一维全为1。
conv = tf.nn.conv2d(x, filter_weight, strides = [1, 2, 2, 1], padding = 'SAME') # padding = 'VALID'
bias = tf.nn.bias_add(conv, biases)
pool = tf.nn.avg_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    convoluted_M = sess.run(bias,feed_dict={x:M})
    pooled_M = sess.run(pool,feed_dict={x:M})
    
    print "convoluted_M: \n", convoluted_M
    print "pooled_M: \n", pooled_M

convoluted_M: 
[[[[ 7.]
   [ 1.]]

  [[-1.]
   [-1.]]]]
pooled_M: 
[[[[ 0.25]
   [ 0.5 ]]

  [[ 1.  ]
   [-2.  ]]]]
